In [1]:
# Imports

import sys
import random
import numpy as np
from sklearn.datasets import load_iris
import pandas as pd
import sklearn.model_selection
from sklearn.preprocessing import StandardScaler

In [2]:
# Declare constants

NUM_INPUT_NODES = 4
NUM_OUTPUT_NODES = 3
NUM_HIDDEN_NODES = 10
NUM_HIDDEN_LAYERS = 1

In [3]:
class Node:
    """Each node has a weight and bias"""
    def __init__(self, weight, bias):
        self.activation = 0
        self.weight = weight
        self.bias = bias

In [4]:
# activation functions

def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def leaky_relu(x, alpha=0.01):
    return max(alpha * x, x)

def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits))  # for numerical stability
    return exp_logits / np.sum(exp_logits, axis=0)

def sigmoid_derivative(x):
    return x * (1 - x)

In [5]:
class MLP:
    "Create a multilayer perceptron"
    def __init__(self, n_in, n_out, n_hidden, hidden_layers):
        self.n_hidden_layers = hidden_layers;
        self.nodes = []
        self.create(n_in, n_out, n_hidden, hidden_layers)

    def create(self, n_in, n_out, n_hidden, hidden_layers):
        nodes = []
        input_layer = [Node(0, 0) for _ in range(n_in)]
        nodes.append(input_layer)
        for i in range(hidden_layers):
            nodes.append([Node(random.uniform(0.0, 0.01), random.uniform(0.0, 0.01)) for _ in range(n_hidden)])
        hidden_layer = [Node(random.uniform(0.0, 0.01), random.uniform(0.0, 0.01)) for _ in range(n_out   )]
        nodes.append(hidden_layer)
        self.nodes = nodes

    def forward_pass(self, X_row):
        # input layer
        for i, node in enumerate(self.nodes[0]):
            node.activation = X_row[i]

        # input layer to hidden layer
        for L in range(1, self.n_hidden_layers + 1):  # note +1 must be used bc the range is not inclusive of upper
            for j in self.nodes[L]:
                j.activation = sigmoid(sum(k.activation * j.weight for k in self.nodes[L - 1]) + j.bias)

        # calculate activations without softmax
        for node in self.nodes[-1]:
            node.activation = sum(k.activation * node.weight for k in self.nodes[-2]) + node.bias

        # apply softmax to output layer activations as a vector
        softmax_output = softmax([node.activation for node in self.nodes[-1]])
        
        # update activations of output layer
        for i, node in enumerate(self.nodes[-1]):
            node.activation = softmax_output[i]

    def back_propogation(self, n):
        """
        n: learning rate
        """
        # calculate values used in the gradient
        # create the gradient

        # hidden layer
        for node in self.nodes[j]:
            k = j - 1
            delta_k = 1  # derivative of E w respect to activation input z
            o_j = 1  # weighted output of the layer
            weight_jk = node.weight + n * delta_k * o_j
            

In [6]:
# for L in range(1, 2): 
#     print(L)
#     for j in mlp.nodes[L]:
#         j.activation = sigmoid(sum(k.activation * j.weight for k in mlp.nodes[L - 1]) + j.bias)
        
# a = [k.activation for k in mlp.nodes[0]]
# b = [j.weight for j in mlp.nodes[1]]
# hidden = [node.bias for node in mlp.nodes[1]]
# b

In [7]:
# Create MLP

mlp = MLP(NUM_INPUT_NODES, NUM_OUTPUT_NODES, NUM_HIDDEN_NODES, NUM_HIDDEN_LAYERS)

In [8]:
iris = load_iris()

In [9]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [10]:
X = iris.data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [11]:
y = iris.target

In [12]:
# Number of classes
num_classes = np.max(y) + 1

# One-hot encode the target variable y
y_one_hot = np.eye(num_classes)[y]

In [13]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.33)

In [14]:
# training loop
epochs = 1
for i in range(0, epochs):
    # 1 epoch includes all data 
    errors = []
    for i, row in enumerate(X_train):
        mlp.forward_pass(X_train[i])
        input = [node.activation for node in mlp.nodes[0]]
        hidden = [node.activation for node in mlp.nodes[1]]
        output = [node.activation for node in mlp.nodes[-1]]
        print(input)
        print(hidden)
        print(output)
        print("------------")
        e = ([node.activation for node in mlp.nodes[-1]] - y[i])**2
        errors.append(e)
    E = sum(errors)/2
    # back propogation algorithm

[1.0380047568006114, -0.1319794793216258, 0.7059208422669494, 0.659038469346772]
[0.5023011204959733, 0.5030331262810058, 0.5034463154633405, 0.5059436685897002, 0.5074887708075192, 0.5067177410945997, 0.505028064343295, 0.5011482109480443, 0.5023498706656037, 0.5058650369988348]
[0.33087284852153015, 0.3346744814802786, 0.3344526699981913]
------------
[-0.5371775589668552, 1.939791417006192, -1.3970639535363667, -1.052179926427139]
[0.5021138093150344, 0.5004249756049232, 0.4999536912447284, 0.5009104478941204, 0.500157358791818, 0.5002527145684283, 0.4988704843694091, 0.5003560472902748, 0.5006141873761832, 0.4992674848187134]
[0.3308996198260978, 0.3346591778897259, 0.33444120228417634]
------------
[0.6745011454696578, -0.36217624558027245, 0.3080588538717274, 0.13250973218556866]
[0.5022154097624396, 0.5018396872490286, 0.501848163753315, 0.5036406296483699, 0.5041342352726141, 0.5037595895386688, 0.5022105239863264, 0.5007857291156537, 0.5015556523132427, 0.5028462101848341]
[0.